# Parameterization of function arguments

There are many use cases where, given a particular function $f(x) \to y$, you would like to pass multiple $x$ of length $n$ instead, getting $y_n$ results. 

In Python this is traditionally done with for loops:

```python
def f(x):
    return x**2

result = [f(_x) for _x in [1, 2, 3]]
```

This can become more problematic when you want to, say parameterize over two arguments:

```python
import itertools as it
def f(x, y):
    return x + y**2

result = [f(_x, _y) for _x, _y in it.product(x, y)]
```

And let's say you wish to include keyword arguments into this; well good luck:

```python
def f(x, y, z):
    return x + y**2 - z*.5

# result = f(1., 2., z=[4., 5., 6.]) ?
```

In [1]:
import numpy as np
import sys
sys.path.insert(0,"../")
# our main import
import turbopanda as turb

print("turbopanda: %s" % turb.__version__)

turbopanda: 0.2.5


## Fear not: There lies a solution

In `turbopanda` we have a useful decorator function called `vectorize` which does precisely this!

There is a catch: when calling a custom function, you must wrap your argument you wish to iterate over using a `turb.Vector` object (which is just an inherited Python list).

## Basic Use Case

A simple use case would be to vectorize a simple float-in float-out function:

In [2]:
from turbopanda import Vector as Vec

In [3]:
@turb.vectorize
def f(x):
    return x**3

f(Vec(2, 4, 6))

[8, 64, 216]

The `Vector` object is important so that `vectorize` knows which parameters to iterate over, rather than using standard `list`, `tuple` or `dict` objects that Python provides by default.

This allows you to pass *normal* list arguments, etc if your function normally uses them:

In [4]:
@turb.vectorize
def g(x, yj):
    # x is a float, y is a list of float
    return [a*x for a in yj]

g(Vec(2., 3.), [3., -1., 2.])

[[6.0, -2.0, 4.0], [9.0, -3.0, 6.0]]

## Combinations of Vectorizations

By Default if more than one parameter contains a `Vector` object, the **product** of the arguments is calculated.

This means if argument `a` has 3 parameters, and argument `b` also has 3 parameters, function $f(a, b, \dots)$ is called 9 times.

In [5]:
@turb.vectorize
def g(x, y, z):
    # x is a float, y is a list of float
    return np.cos(x) + np.sin(y) * -.5 * np.pi * z

g(Vec(1., np.pi, -np.pi), np.pi / 2, Vec(0., np.pi))

[0.5403023058681397,
 -4.39449989467654,
 -1.0,
 -5.934802200544679,
 -1.0,
 -5.934802200544679]

This expands up to $\inf$ arguments, although watch out as you don't want your computer to denotate from the parameterization explosion.

## Incorporating keyword arguments

For long functions with many *default parameters*, it is highly useful to use keyword arguments just to pick-and-choose which parameters to tweak. For one of these parameters, there are a number of use cases where vectorizing over this parameter could yield some useful information.

In [6]:
g(2., np.pi, z=Vec(3., 4.))

[-0.41614683654714296, -0.4161468365471432]

This also works if other keyword arguments you specify are not vectorized:

In [7]:
g(x=2., y=np.pi, z=Vec(3., 4.))

[-0.41614683654714296, -0.4161468365471432]

## Automatic Parallel Backend

By default `vectorize` has a parallel argument which is set to `False`, but understandably with large numbers of arguments in a potentially slow function, we would like to parallelize calculations on the function, given the key assumption that each set of arguments are independent from one another. 

The decorator has a `parallel` keyword which can be set:

In [8]:
import pandas as pd

In [20]:
@turb.vectorize(parallel=True)
def h(x, z, a, q):
    return x*z - a*q

h(Vec(*np.arange(10)), 2., -3., Vec(np.arange(-15, 15, 1.5)))

[array([-45. , -40.5, -36. , -31.5, -27. , -22.5, -18. , -13.5,  -9. ,
         -4.5,   0. ,   4.5,   9. ,  13.5,  18. ,  22.5,  27. ,  31.5,
         36. ,  40.5]),
 array([-43. , -38.5, -34. , -29.5, -25. , -20.5, -16. , -11.5,  -7. ,
         -2.5,   2. ,   6.5,  11. ,  15.5,  20. ,  24.5,  29. ,  33.5,
         38. ,  42.5]),
 array([-41. , -36.5, -32. , -27.5, -23. , -18.5, -14. ,  -9.5,  -5. ,
         -0.5,   4. ,   8.5,  13. ,  17.5,  22. ,  26.5,  31. ,  35.5,
         40. ,  44.5]),
 array([-39. , -34.5, -30. , -25.5, -21. , -16.5, -12. ,  -7.5,  -3. ,
          1.5,   6. ,  10.5,  15. ,  19.5,  24. ,  28.5,  33. ,  37.5,
         42. ,  46.5]),
 array([-37. , -32.5, -28. , -23.5, -19. , -14.5, -10. ,  -5.5,  -1. ,
          3.5,   8. ,  12.5,  17. ,  21.5,  26. ,  30.5,  35. ,  39.5,
         44. ,  48.5]),
 array([-35. , -30.5, -26. , -21.5, -17. , -12.5,  -8. ,  -3.5,   1. ,
          5.5,  10. ,  14.5,  19. ,  23.5,  28. ,  32.5,  37. ,  41.5,
         46. ,  50.5]),
 arr

Note in the above example that I **unpacked** the numpy array into the `Vector` constructor such that those numbers were in list form.

`Vector` will take any object which would normally be represented as a parameter, but it will **NOT** unpack vectors/matrices for you, assuming that you want to pass this as a whole to the function:

In [18]:
try:
    h(Vec(np.arange(10), 2), 2., -3., Vec(np.arange(-15, 15, 1.5)))
except ValueError as e:
    print(e)

operands could not be broadcast together with shapes (10,) (20,) 


In the above case, the first argument in `Vec` passes the whole numpy array as the first argument to the function, rather than 10 individual numbers.

Just be careful, particular in **product** use cases that what your result is, actually makes sense.